In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.cuda.amp import GradScaler, autocast

# ⚙️ Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 📁 Paths
TRAIN_DIR = 'D:/Projects/iit_ropar_project/train'
TEST_DIR = 'D:/Projects/iit_ropar_project/test'
TRAIN_CSV = 'D:/Projects/iit_ropar_project/train_labels.csv'
TEST_CSV = 'D:/Projects/iit_ropar_project/test_ids.csv'
SUBMISSION_CSV = 'D:/Projects/iit_ropar_project/sample_submission.csv'

IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCHS = 30
LABELS = ['Alluvial soil', 'Black Soil', 'Clay soil', 'Red soil']
label2idx = {label: i for i, label in enumerate(LABELS)}
idx2label = {i: label for label, i in label2idx.items()}

# 🧱 Dataset Class
class SoilDataset(Dataset):
    def __init__(self, df, img_dir, labels=True, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.has_labels = labels

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]['image_id']
        img_path = os.path.join(self.img_dir, img_id)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.has_labels:
            label = label2idx[self.df.iloc[idx]['label']]
            return image, label
        else:
            return image, img_id

# 🔧 Transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.GaussianBlur(3),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
    transforms.RandomErasing()
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# 📊 Load Data
df = pd.read_csv(TRAIN_CSV)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
train_dataset = SoilDataset(train_df, TRAIN_DIR, labels=True, transform=train_transform)
val_dataset = SoilDataset(val_df, TRAIN_DIR, labels=True, transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# 🧠 Models
class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.AdaptiveAvgPool2d((1, 1))
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, NUM_CLASSES)
        )

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

def get_model(base='mobilenet'):
    if base == 'mobilenet':
        model = models.mobilenet_v2(pretrained=True)
        model.classifier[1] = nn.Linear(model.last_channel, NUM_CLASSES)
    elif base == 'resnet':
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    return model

# 🏋️ Training Function
def train_model(model, name):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    scaler = GradScaler()

    best_acc, stop_counter, patience = 0, 0, 5

    for epoch in range(EPOCHS):
        model.train()
        correct, total = 0, 0
        loop = tqdm(train_loader, desc=f"[{name}] Epoch {epoch+1}/{EPOCHS}")
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            loop.set_postfix(loss=loss.item(), acc=100 * correct / total)

        val_acc = evaluate(model)
        print(f"Epoch {epoch+1}: Val Acc: {val_acc:.2f}%")
        scheduler.step()

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), f"{name}_best.pth")
            stop_counter = 0
        else:
            stop_counter += 1
            if stop_counter >= patience:
                print(f"Early stopping for {name}")
                break

    print(f"✅ Best Validation Accuracy for {name}: {best_acc:.2f}%")
    return model

# 📈 Evaluation
def evaluate(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total

# 🔮 Predict Test Set
def predict_test(models, weights=None):
    for model in models:
        model.eval()

    test_df = pd.read_csv(TEST_CSV)
    test_dataset = SoilDataset(test_df, TEST_DIR, labels=False, transform=val_transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

    all_preds, all_ids = [], []

    with torch.no_grad():
        for images, img_ids in tqdm(test_loader, desc="Predicting Test Set"):
            images = images.to(device)

            outputs = []
            for i, model in enumerate(models):
                model.to(device)
                out = model(images)
                softmax_out = torch.softmax(out, dim=1)
                if weights:
                    softmax_out *= weights[i]
                outputs.append(softmax_out)
                model.to('cpu')
                torch.cuda.empty_cache()

            avg_output = sum(outputs) / (sum(weights) if weights else len(models))
            preds = torch.argmax(avg_output, dim=1).cpu().numpy()
            all_preds.extend([idx2label[i] for i in preds])
            all_ids.extend(img_ids)

    submission = pd.DataFrame({'image_id': all_ids, 'label': all_preds})
    submission.to_csv(SUBMISSION_CSV, index=False)
    print(f"📁 Saved predictions to {SUBMISSION_CSV}")

# 🚀 Main
if __name__ == '__main__':
    model1 = train_model(CustomCNN(), 'CustomCNN')
    model2 = train_model(get_model('mobilenet'), 'MobileNetV2')
    model3 = train_model(get_model('resnet'), 'ResNet50')

    model1.load_state_dict(torch.load("CustomCNN_best.pth"))
    model2.load_state_dict(torch.load("MobileNetV2_best.pth"))
    model3.load_state_dict(torch.load("ResNet50_best.pth"))

    predict_test([model1.to(device), model2.to(device), model3.to(device)], weights=[0.2, 0.4, 0.4])


Device: cuda


C:\Users\Chirag Bansal\AppData\Local\Temp\ipykernel_22808\868957484.py:117: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
[CustomCNN] Epoch 1/30:   0%|          | 0/31 [00:00<?, ?it/s]C:\Users\Chirag Bansal\AppData\Local\Temp\ipykernel_22808\868957484.py:129: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
[CustomCNN] Epoch 1/30: 100%|██████████| 31/31 [00:16<00:00,  1.90it/s, acc=24.3, loss=1.37]


Epoch 1: Val Acc: 61.63%


[CustomCNN] Epoch 2/30: 100%|██████████| 31/31 [00:14<00:00,  2.20it/s, acc=55.1, loss=1.35]


Epoch 2: Val Acc: 64.90%


...

Epoch 17: Val Acc: 97.14%


[ResNet50] Epoch 18/30: 100%|██████████| 31/31 [00:15<00:00,  1.94it/s, acc=99.1, loss=0.369]


Epoch 18: Val Acc: 96.73%
Early stopping for ResNet50
✅ Best Validation Accuracy for ResNet50: 97.14%


C:\Users\Chirag Bansal\AppData\Local\Temp\ipykernel_22808\868957484.py:213: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load("CustomCNN_best.p

📁 Saved predictions to D:/Projects/iit_ropar_project/sample_submission.csv
